In [ ]:
%matplotlib inline
import os
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import xarray as xr

In [2]:
import salem

In [3]:
from glob import glob

### past 4 months, 7 days chunks 

In [4]:
lfiles = glob('/Volumes/GDATA/data/CRW/BAA7dMAX/*.nc')

In [ ]:
lfiles.sort()

In [ ]:
lfiles

### reads in the BAA 7 days max

In [ ]:
dset = xr.open_mfdataset(lfiles)

In [ ]:
dset

### calculates the max over time

In [ ]:
dset = dset.max('time')

In [ ]:
dset

In [ ]:
dset_rolled = dset.assign_coords(lon=(dset.lon % 360)).roll(lon=(dset.dims['lon'] // 2))

In [ ]:
plt.imshow(dset['CRW_BAA_max7d'].data)

In [ ]:
plt.imshow(dset_rolled['CRW_BAA_max7d'].data)

### loads in memory

In [ ]:
dset.load()

### reads in the masks

In [ ]:
masks = xr.open_dataset('../data/reef_mask/compiled_5km_reef_mask_for_edit_20171113_buffer4pix.nc')

In [ ]:
masks

In [ ]:
plt.imshow(masks['land_mask'].data)

In [ ]:
masks_rolled = masks.assign_coords(lon=(masks.lon % 360)).roll(lon=(masks.dims['lon'] // 2))

In [ ]:
plt.imshow(masks_rolled['land_mask'].data)

In [ ]:
land_mask = ~(masks_rolled['land_mask'].data.astype(np.bool))

In [ ]:
land_mask = land_mask.astype(np.int32)

In [ ]:
land_mask

In [ ]:
plt.imshow(land_mask); plt.colorbar()

In [ ]:
masks_rolled['land_mask '] = (('lat','lon'), land_mask)

In [ ]:
dset_rolled['land_mask'] = masks_rolled['land_mask']

In [ ]:
dset_rolled['reef_mask'] = masks_rolled['reef_mask']

In [ ]:
if os.path.exists('../outputs/tmp.nc'):
    os.remove('../outputs/tmp.nc')

In [ ]:
dset_rolled.to_netcdf('../outputs/tmp.nc')

### reads the EEZ shapefile

In [ ]:
shp_path = "/Users/nicolasf/drives/auck_projects/ICU14101/Working/ICU Water Watch/GIS map files"

In [ ]:
shp = salem.read_shapefile(os.path.join(shp_path, "ALL_ICU_countries.shp"))

In [ ]:
shp.plot()

In [ ]:
shp.head()

In [ ]:
dset_rolled.to_netcdf('../outputs/tmp.nc')

In [ ]:
dset_salem = salem.open_xr_dataset('../outputs/tmp.nc')

In [ ]:
dset_salem

In [ ]:
shp.COUNTRYNAM.unique()

In [ ]:
country_name = 'Fiji'

In [ ]:
sh_country = shp.query("COUNTRYNAM == '{}'".format(country_name))

In [ ]:
sh_country

In [ ]:
sh_country.to_file('./shapes_{}'.format(country_name))

In [ ]:
import fiona

In [ ]:
sh_country = fiona.open('./shapes_{}'.format(country_name))

In [ ]:
with fiona.open('./shapes_{}_converted.shp'.format(country_name), 'w', 'ESRI Shapefile', sh_country.schema.copy(), crs = sh_country.crs) as output:
    for shape in sh_country:
        if shape['properties']['min_x'] < 0: 
            shape['properties']['min_x'] += 360 
        if shape['properties']['max_x'] < 0: 
            shape['properties']['max_x'] += 360
        if (len(shape['geometry']['coordinates']) == 1):
            coords = shape['geometry']['coordinates']
            coords_mod = coords.copy()
            for j in range(len(coords[0])): 
                lon, lat = coords[0][j]
                if lon < 0: 
                    lon += 360
                coords_mod[0][j] = (lon, lat)
            shape['geometry']['coordinates'] = coords_mod  
        else: 
            for i in range(len(shape['geometry']['coordinates'])): 
                coords = shape['geometry']['coordinates'][i]
                coords_mod = coords.copy()
                for j in range(len(coords[0])): 
                    lon, lat = coords[0][j]
                    if lon < 0: 
                        lon += 360
                    coords_mod[0][j] = (lon, lat)
                shape['geometry']['coordinates'][i] = coords_mod  
        output.write(shape)

In [ ]:
sh_country = salem.read_shapefile('./shapes_{}_converted.shp'.format(country_name))

In [ ]:
ds_subset = dset_salem.salem.subset(shape=sh_country, margin=2)

In [ ]:
plt.imshow(ds_subset['CRW_BAA_max7d'].data)

In [ ]:
ds_subset_roi = ds_subset.salem.roi(shape=sh_country)

In [ ]:
plt.imshow(ds_subset_roi['CRW_BAA_max7d'].data)

In [ ]:
ds_subset_roi

In [ ]:
ds_subset['reef_mask'].plot()

In [ ]:
sh_country.plot()